In [1]:
import boto3
# retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  vivekp


In [2]:
import boto3
import json
import time

# Initialize clients
rekognition_client = boto3.client('rekognition')
sqs_client = boto3.client('sqs')

# Configuration
bucket_name = 'vivekp'
video_file = 'MORNING ALARM BE LIKE (Animation Meme).mp4'
sns_topic_arn = 'arn:aws:sns:ap-south-1:381492233747:shot_detect'
iam_role_arn = 'arn:aws:iam::381492233747:role/vivek_role'
sqs_queue_url = 'https://sqs.ap-south-1.amazonaws.com/381492233747/shot_detection'

# Start label detection
response = rekognition_client.start_label_detection(
    Video={'S3Object': {'Bucket': bucket_name, 'Name': video_file}},
    NotificationChannel={'SNSTopicArn': sns_topic_arn, 'RoleArn': iam_role_arn},
    JobTag='DetectingLabels'
)
job_id = response['JobId']
print(f'Started label detection job with ID: {job_id}')

# Poll the SQS queue for completion status
def get_job_status(job_id):
    while True:
        messages = sqs_client.receive_message(
            QueueUrl=sqs_queue_url,
            MaxNumberOfMessages=10,
            WaitTimeSeconds=10
        )

        if 'Messages' in messages:
            for message in messages['Messages']:
                notification = json.loads(message['Body'])
                rek_message = json.loads(notification['Message'])

                if rek_message['JobId'] == job_id:
                    if rek_message['Status'] == 'SUCCEEDED':
                        print('Job succeeded.')
                        return True
                    elif rek_message['Status'] == 'FAILED':
                        print('Job failed.')
                        return False

                sqs_client.delete_message(
                    QueueUrl=sqs_queue_url,
                    ReceiptHandle=message['ReceiptHandle']
                )
        else:
            print('Waiting for job completion...')
            time.sleep(5)

job_completed = get_job_status(job_id)

if not job_completed:
    raise Exception('Video analysis job failed.')

# Retrieve label detection results
result = rekognition_client.get_label_detection(
    JobId=job_id,
    MaxResults=10,
    SortBy='TIMESTAMP'
)

for label in result['Labels']:
    print(f"Timestamp: {label['Timestamp']}")
    print(f"Label: {label['Label']['Name']}")
    print(f"Confidence: {label['Label']['Confidence']:.2f}%")
    print("Instances:")
    for instance in label['Label']['Instances']:
        print(f"  BoundingBox: {instance['BoundingBox']}")
        print(f"  Confidence: {instance['Confidence']:.2f}%")
    print("-" * 60)


Started label detection job with ID: 4fa9a20d27eee161b07ca2ba4f5d0d51275f23e7c6f22890384888b26d70eab0
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...
Waiting for job completion...


KeyboardInterrupt: 